<a href="https://colab.research.google.com/github/zbatzioutilburg/Thesis/blob/main/Thesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -q -U keras-tuner

In [ ]:
pip install PyPortfolioOpt

In [ ]:
### Import libraries.
import math
import pandas as pd
import numpy as np
import tensorflow as tf
import IPython
import IPython.display
import keras_tuner as kt
import random
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from functools import reduce
import pypfopt
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

In [99]:
random.seed(1)

In [100]:
### Import data csv files, set index as datetime
### and calculate logarithmic returns.

#Macroeconomic variables

CPIAUCSL = pd.read_csv('CPIAUCSL.csv', sep = ',', index_col = 0)
CPIAUCSL.index = pd.to_datetime(CPIAUCSL.index)
CPIAUCSL = np.log(CPIAUCSL.CPIAUCSL) - np.log(CPIAUCSL.CPIAUCSL.shift(1))

CSUSHPISA = pd.read_csv('CSUSHPISA.csv', sep = ',', index_col = 0)
CSUSHPISA.index = pd.to_datetime(CSUSHPISA.index)
CSUSHPISA = np.log(CSUSHPISA.CSUSHPISA) - np.log(CSUSHPISA.CSUSHPISA.shift(1))

IQ = pd.read_csv('IQ.csv', sep = ',', index_col = 0)
IQ.index = pd.to_datetime(IQ.index)
IQ = np.log(IQ.IQ) - np.log(IQ.IQ.shift(1))

IR = pd.read_csv('IR.csv', sep = ',', index_col = 0)
IR.index = pd.to_datetime(IR.index)
IR = np.log(IR.IR) - np.log(IR.IR.shift(1))

MCOILWTICO = pd.read_csv('MCOILWTICO.csv', sep = ',', index_col = 0)
MCOILWTICO.index = pd.to_datetime(MCOILWTICO.index)
MCOILWTICO = np.log(MCOILWTICO.MCOILWTICO) - np.log(MCOILWTICO.MCOILWTICO.shift(1))

MICH = pd.read_csv('MICH.csv', sep = ',', index_col = 0)
MICH.index = pd.to_datetime(MICH.index)
MICH = np.log(MICH.MICH) - np.log(MICH.MICH.shift(1))

PPIACO = pd.read_csv('PPIACO.csv', sep = ',', index_col = 0)
PPIACO.index = pd.to_datetime(PPIACO.index)
PPIACO = np.log(PPIACO.PPIACO) - np.log(PPIACO.PPIACO.shift(1))

PSAVERT = pd.read_csv('PSAVERT.csv', sep = ',', index_col = 0)
PSAVERT.index = pd.to_datetime(PSAVERT.index)
PSAVERT = np.log(PSAVERT.PSAVERT) - np.log(PSAVERT.PSAVERT.shift(1))

TB3MS = pd.read_csv('TB3MS.csv', sep = ',', index_col = 0)
TB3MS.index = pd.to_datetime(TB3MS.index)
TB3MS = np.log(TB3MS.TB3MS) - np.log(TB3MS.TB3MS.shift(1))

TWEXBMTH = pd.read_csv('TWEXBMTH.csv', sep = ',', index_col = 0)
TWEXBMTH.index = pd.to_datetime(TWEXBMTH.index)
TWEXBMTH = np.log(TWEXBMTH.TWEXBMTH) - np.log(TWEXBMTH.TWEXBMTH.shift(1))

USD1MTD156N = pd.read_csv('USD1MTD156N.csv', sep = ',', index_col = 0)
USD1MTD156N.index = pd.to_datetime(USD1MTD156N.index)


#Financial variables
NAESX = pd.read_csv('NAESX.csv', sep = ',', index_col = 0)
NAESX.index = pd.to_datetime(NAESX.index)
NAESX = NAESX.drop(columns = ['Open', 'High', 'Low', 'Close', 'Volume'])
NAESX.columns = ['NAESX Adj. Close']
NAESX = np.log(NAESX['NAESX Adj. Close']) - np.log(NAESX['NAESX Adj. Close'].shift(1))

VBMFX = pd.read_csv('VBMFX.csv', sep = ',', index_col = 0)
VBMFX.index = pd.to_datetime(VBMFX.index)
VBMFX = VBMFX.drop(columns = ['Open', 'High', 'Low', 'Close', 'Volume'])
VBMFX.columns = ['VBMFX Adj. Close']
VBMFX = np.log(VBMFX['VBMFX Adj. Close']) - np.log(VBMFX['VBMFX Adj. Close'].shift(1))

VEIEX = pd.read_csv('VEIEX.csv', sep = ',', index_col = 0)
VEIEX.index = pd.to_datetime(VEIEX.index)
VEIEX = VEIEX.drop(columns = ['Open', 'High', 'Low', 'Close', 'Volume'])
VEIEX.columns = ['VEIEX Adj. Close']
VEIEX = np.log(VEIEX['VEIEX Adj. Close']) - np.log(VEIEX['VEIEX Adj. Close'].shift(1))

VFINX = pd.read_csv('VFINX.csv', sep = ',', index_col = 0)
VFINX.index = pd.to_datetime(VFINX.index)
VFINX = VFINX.drop(columns = ['Open', 'High', 'Low', 'Close', 'Volume'])
VFINX.columns = ['VFINX Adj. Close']
VFINX = np.log(VFINX['VFINX Adj. Close']) - np.log(VFINX['VFINX Adj. Close'].shift(1))

VGSLX = pd.read_csv('VGSLX.csv', sep = ',', index_col = 0)
VGSLX.index = pd.to_datetime(VGSLX.index)
VGSLX = VGSLX.drop(columns = ['Open', 'High', 'Low', 'Close', 'Volume'])
VGSLX.columns = ['VGSLX Adj. Close']
VGSLX = np.log(VGSLX['VGSLX Adj. Close']) - np.log(VGSLX['VGSLX Adj. Close'].shift(1))

VGTSX = pd.read_csv('VGTSX.csv', sep = ',', index_col = 0)
VGTSX.index = pd.to_datetime(VGTSX.index)
VGTSX = VGTSX.drop(columns = ['Open', 'High', 'Low', 'Close', 'Volume'])
VGTSX.columns = ['VGTSX Adj. Close']
VGTSX = np.log(VGTSX['VGTSX Adj. Close']) - np.log(VGTSX['VGTSX Adj. Close'].shift(1))

In [101]:
### The Libor dataset has daily values.
### They will be converted to monthly values, averaging per month.
### The Libor dataset index is converted to datetime.

USD1MTD156N.USD1MTD156N = USD1MTD156N.USD1MTD156N.replace('.', '')
USD1MTD156N.index = pd.to_datetime(USD1MTD156N.index)
USD1MTD156N.USD1MTD156N = pd.to_numeric(USD1MTD156N.USD1MTD156N)
USD1MTD156N = USD1MTD156N.resample('1MS').mean()
USD1MTD156N = np.log(USD1MTD156N.USD1MTD156N) - np.log(USD1MTD156N.USD1MTD156N.shift(1))

In [102]:
### Prepare the datasets.
input_variables = [CPIAUCSL, CSUSHPISA, IQ, IR,
                   MCOILWTICO, MICH, PPIACO, PSAVERT,
                   TB3MS, TWEXBMTH, USD1MTD156N]

output_variables = [NAESX, VBMFX, VEIEX, VFINX, VGSLX, VGTSX]

X = reduce(lambda left, right : pd.merge(left, right,
                                         left_index = True, right_index = True,
                                         how = 'outer'), input_variables)

y = reduce(lambda left, right : pd.merge(left, right,
                                         left_index = True, right_index = True,
                                         how = 'outer'), output_variables)

X = X['2002-01-01' : '2019-12-01']
y = y['2002-01-01' : '2019-12-01']
y = y.fillna(0.0)

In [103]:
### Split the data into train, validation and test sets,
### with analogy of 70%, 15% and 15%, respectively.
X_train = X[0 : int(0.7*len(X))]
X_val = X[int(0.7*len(X)) : int(0.85*len(X))]
X_test = X[int(0.85*len(X)) : ]
y_train = y[0: int(0.7*len(y))]
y_val = y[int(0.7*len(y)) : int(0.85*len(y))]
y_test = y[int(0.85*len(y)) : ]

In [104]:
### Convert to numpy arrays.
X_train = X_train.values
X_val = X_val.values
X_test = X_test.values
y_train = y_train.values
y_val = y_val.values
y_test = y_test.values

In [105]:
### Data standardization.
X_train = (X_train - X_train.mean()) / X_train.std()
X_val = (X_val - X_train.mean()) / X_train.std()
X_test = (X_test - X_train.mean()) / X_train.std()

Multi-Layer Perceptron (MLP Baseline Model - 1 hidden hayer)

In [106]:
### Create MLP model with the use of a function.
def mlp_builder(hp):
  hp_units = hp.Int('units', min_value = 10, max_value = 100, step = 10)
  hp_learning_rate = hp.Choice('learning_rate',
                               values = [1e-1, 1e-2, 1e-3, 1e-4])
  hp_activation = hp.Choice('activation',
                            values = ['relu', 'tanh', 'sigmoid'])
  model = tf.keras.Sequential()
  model.add((keras.layers.Dense(units = hp_units,
                                     activation = hp_activation,
                                     input_dim = X_train.shape[1])))
  model.add(keras.layers.Dense(y_train.shape[1]))

  model.compile(optimizer = Adam(learning_rate = hp_learning_rate),
                loss = 'mse',
                metrics = tf.keras.metrics.RootMeanSquaredError())
  return model

In [107]:
### Instantiate the tuner and perform hyperparameter tuning.
tuner = kt.Hyperband(mlp_builder,
                     objective = 'val_loss',
                     max_epochs = 10,
                     directory = 'keras_tuner_dir',
                     project_name = 'keras_mlp',
                     overwrite = True)

In [108]:
### Early Stopping definition.
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 5)

In [109]:
### Hyperparameters search for 5 epochs.
tuner.search(X_train, y_train,
             epochs = 5, validation_data = (X_val, y_val),
             callbacks = [stop_early])

Trial 30 Complete [00h 00m 02s]
val_loss: 0.11313724517822266

Best val_loss So Far: 0.0018050395883619785
Total elapsed time: 00h 00m 38s
INFO:tensorflow:Oracle triggered exit


In [110]:
# Get the optimal hyperparameters.
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

In [111]:
print(f"""
The hyperparameter search for MLP is complete.
The optimal number of units in the first hidden
layer is {best_hps.get('units')},
the optimal activation function is {best_hps.get('activation')}
and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")


The hyperparameter search for MLP is complete.
The optimal number of units in the first hidden
layer is 80,
the optimal activation function is sigmoid
and the optimal learning rate for the optimizer
is 0.01.



In [112]:
### Find the optimal number of epochs to train the model with the hyperparameters
### obtained from the search. Build the model with the optimal hyperparameters
### and train it on the data for 200 epochs.
model = tuner.hypermodel.build(best_hps)

history = model.fit(X_train, y_train,
                    epochs=200, validation_data = (X_val, y_val))

val_rmse_per_epoch = history.history['val_root_mean_squared_error']

best_epoch = val_rmse_per_epoch.index(min(val_rmse_per_epoch))

print('Best epoch: %d' % (best_epoch,))

Epoch 1/200
5/5 [==============================] - 1s 37ms/step - loss: 0.2111 - root_mean_squared_error: 0.4594 - val_loss: 0.1247 - val_root_mean_squared_error: 0.3532
Epoch 2/200
5/5 [==============================] - 0s 9ms/step - loss: 0.1118 - root_mean_squared_error: 0.3344 - val_loss: 0.0259 - val_root_mean_squared_error: 0.1609
Epoch 3/200
5/5 [==============================] - 0s 11ms/step - loss: 0.0333 - root_mean_squared_error: 0.1824 - val_loss: 0.0457 - val_root_mean_squared_error: 0.2137
Epoch 4/200
5/5 [==============================] - 0s 8ms/step - loss: 0.0347 - root_mean_squared_error: 0.1864 - val_loss: 0.0136 - val_root_mean_squared_error: 0.1165
Epoch 5/200
5/5 [==============================] - 0s 9ms/step - loss: 0.0148 - root_mean_squared_error: 0.1216 - val_loss: 0.0166 - val_root_mean_squared_error: 0.1287
Epoch 6/200
5/5 [==============================] - 0s 9ms/step - loss: 0.0150 - root_mean_squared_error: 0.1226 - val_loss: 0.0039 - val_root_mean_square

In [113]:
### Pass the optimal hyperparameters to hypermodel.
hypermodel = tuner.hypermodel.build(best_hps)

### Retrain the model with the optimal number of epochs.
hypermodel.fit(X_train, y_train,
               epochs=best_epoch,
               validation_data = (X_val, y_val),
               shuffle = False)

Epoch 1/28
5/5 [==============================] - 1s 39ms/step - loss: 0.3882 - root_mean_squared_error: 0.6230 - val_loss: 0.0859 - val_root_mean_squared_error: 0.2930
Epoch 2/28
5/5 [==============================] - 0s 11ms/step - loss: 0.1538 - root_mean_squared_error: 0.3922 - val_loss: 0.1370 - val_root_mean_squared_error: 0.3701
Epoch 3/28
5/5 [==============================] - 0s 8ms/step - loss: 0.0845 - root_mean_squared_error: 0.2907 - val_loss: 0.0150 - val_root_mean_squared_error: 0.1226
Epoch 4/28
5/5 [==============================] - 0s 8ms/step - loss: 0.0369 - root_mean_squared_error: 0.1920 - val_loss: 0.0550 - val_root_mean_squared_error: 0.2345
Epoch 5/28
5/5 [==============================] - 0s 7ms/step - loss: 0.0376 - root_mean_squared_error: 0.1939 - val_loss: 0.0105 - val_root_mean_squared_error: 0.1024
Epoch 6/28
5/5 [==============================] - 0s 9ms/step - loss: 0.0143 - root_mean_squared_error: 0.1195 - val_loss: 0.0135 - val_root_mean_squared_erro

In [114]:
### Predicted output outcomes using X_test.
y_pred = hypermodel.predict(X_test)

In [115]:
### Evaluate hypermodel using test sets.
eval_result = hypermodel.evaluate(X_test, y_test)
print("[test loss, test rmse]:", eval_result)

2/2 [==============================] - 0s 8ms/step - loss: 0.0012 - root_mean_squared_error: 0.0350
[test loss, test rmse]: [0.001223293598741293, 0.03497561439871788]


In [116]:
### Find the Root Mean Squared Error
RMSE = math.sqrt(np.square(np.subtract(y_test, y_pred)).mean())
print('RMSE - MLP: ' + str(RMSE))

RMSE - MLP: 0.034975612261528906


Long Short-Term Memory Model (LSTM - 1 hidden layer)

In [117]:
### Reshape the train, val, test sets,
### since LSTMs require 3-dimensional sets.
### (samples, time steps, features)
#X.shape = (N sequence, length of time series, N input features)
#y.shape = (N sequence, length of time series, N targets)
#----------> We choose 1 time step for the sequence.
X_train_2 = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_val_2 = X_val.reshape(X_val.shape[0], 1, X_val.shape[1])
X_test_2 = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])
y_train_2 = y_train.reshape(y_train.shape[0], 1, y_train.shape[1])
y_val_2 = y_val.reshape(y_val.shape[0], 1, y_val.shape[1])
y_test_2 = y_test.reshape(y_test.shape[0], 1, y_test.shape[1])

In [118]:
### Create LSTM model with the use of a function.
def build_lstm(hp):
  hp_units = hp.Int('units', min_value = 10, max_value = 100, step = 10)
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-1, 1e-2, 1e-3, 1e-4])
  hp_activation = hp.Choice('activation', values = ['relu', 'tanh', 'sigmoid'])
  model = tf.keras.Sequential()
  model.add((keras.layers.LSTM(units = hp_units,
                               activation = hp_activation,
                               input_shape = (X_train_2.shape[1],
                                              X_train_2.shape[2]),
                               return_sequences = True)))
  model.add(keras.layers.Dense(y_train_2.shape[2]))

  model.compile(optimizer = Adam(learning_rate = hp_learning_rate),
                loss = 'mse',
                metrics = tf.keras.metrics.RootMeanSquaredError())
  return model

In [120]:
### Instantiate the tuner and perform hyperparameter tuning.
lstm_tuner = kt.Hyperband(build_lstm,
                     objective = 'val_loss',
                     max_epochs = 10,
                     directory = 'keras_tuner_dir',
                     project_name = 'keras_lstm',
                     overwrite = True)

In [121]:
### Early Stopping definition
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 5)

In [122]:
### Hyperparameters search for 5 epochs
lstm_tuner.search(X_train_2, y_train_2,
             epochs = 5, validation_data = (X_val_2, y_val_2),
              callbacks = [stop_early])

Trial 30 Complete [00h 00m 05s]
val_loss: 0.001718671526759863

Best val_loss So Far: 0.0013283481821417809
Total elapsed time: 00h 01m 30s
INFO:tensorflow:Oracle triggered exit


In [123]:
# Get the optimal hyperparameters
lstm_best_hps = lstm_tuner.get_best_hyperparameters(num_trials=1)[0]

In [124]:
print(f"""
The hyperparameter search for LSTM is complete.
The optimal number of units in the first LSTM-connected
layer is {lstm_best_hps.get('units')},
the optimal activation is {lstm_best_hps.get('activation')}
and the optimal learning rate for the optimizer
is {lstm_best_hps.get('learning_rate')}
""")


The hyperparameter search for LSTM is complete.
The optimal number of units in the first LSTM-connected
layer is 80,
the optimal activation is tanh
and the optimal learning rate for the optimizer
is 0.01



In [125]:
### Find the optimal number of epochs to train the model with the hyperparameters
### obtained from the search.
### Build the model with the optimal hyperparameters and
### train it on the data for 100 epochs.
model = lstm_tuner.hypermodel.build(lstm_best_hps)

history = model.fit(X_train_2, y_train_2,
                    epochs=100, validation_data = (X_val_2, y_val_2))

val_rmse_per_epoch = history.history['val_root_mean_squared_error']

best_epoch = val_rmse_per_epoch.index(min(val_rmse_per_epoch))

print('Best epoch: %d' % (best_epoch,))

Epoch 1/100
5/5 [==============================] - 3s 139ms/step - loss: 0.0062 - root_mean_squared_error: 0.0788 - val_loss: 0.0017 - val_root_mean_squared_error: 0.0413
Epoch 2/100
5/5 [==============================] - 0s 11ms/step - loss: 0.0040 - root_mean_squared_error: 0.0635 - val_loss: 0.0014 - val_root_mean_squared_error: 0.0375
Epoch 3/100
5/5 [==============================] - 0s 10ms/step - loss: 0.0036 - root_mean_squared_error: 0.0599 - val_loss: 0.0015 - val_root_mean_squared_error: 0.0386
Epoch 4/100
5/5 [==============================] - 0s 10ms/step - loss: 0.0028 - root_mean_squared_error: 0.0531 - val_loss: 0.0014 - val_root_mean_squared_error: 0.0378
Epoch 5/100
5/5 [==============================] - 0s 10ms/step - loss: 0.0024 - root_mean_squared_error: 0.0493 - val_loss: 0.0013 - val_root_mean_squared_error: 0.0365
Epoch 6/100
5/5 [==============================] - 0s 11ms/step - loss: 0.0023 - root_mean_squared_error: 0.0478 - val_loss: 0.0015 - val_root_mean_s

In [126]:
### Pass the optimal hyperparameters to hypermodel.
hypermodel_2 = lstm_tuner.hypermodel.build(lstm_best_hps)

### Retrain the model with the optimal number of epochs.

hypermodel_2.fit(X_train_2, y_train_2,
                 epochs = best_epoch,
                 validation_data = (X_val_2, y_val_2),
                 shuffle = False)

Epoch 1/9
5/5 [==============================] - 3s 140ms/step - loss: 0.0050 - root_mean_squared_error: 0.0709 - val_loss: 0.0015 - val_root_mean_squared_error: 0.0392
Epoch 2/9
5/5 [==============================] - 0s 10ms/step - loss: 0.0057 - root_mean_squared_error: 0.0757 - val_loss: 0.0014 - val_root_mean_squared_error: 0.0374
Epoch 3/9
5/5 [==============================] - 0s 11ms/step - loss: 0.0034 - root_mean_squared_error: 0.0584 - val_loss: 0.0016 - val_root_mean_squared_error: 0.0405
Epoch 4/9
5/5 [==============================] - 0s 10ms/step - loss: 0.0028 - root_mean_squared_error: 0.0533 - val_loss: 0.0015 - val_root_mean_squared_error: 0.0382
Epoch 5/9
5/5 [==============================] - 0s 10ms/step - loss: 0.0024 - root_mean_squared_error: 0.0491 - val_loss: 0.0014 - val_root_mean_squared_error: 0.0380
Epoch 6/9
5/5 [==============================] - 0s 11ms/step - loss: 0.0022 - root_mean_squared_error: 0.0467 - val_loss: 0.0013 - val_root_mean_squared_error

In [127]:
### Predicted values using X_test_2.
y_pred_lstm = hypermodel_2.predict(X_test_2)

In [128]:
### Evaluate the hypermodel using the test sets.
eval_result_lstm = hypermodel_2.evaluate(X_test_2, y_test_2)
print("[test loss, test mse]:", eval_result_lstm)

2/2 [==============================] - 1s 5ms/step - loss: 0.0012 - root_mean_squared_error: 0.0353
[test loss, test mse]: [0.001247358974069357, 0.03531797230243683]


In [129]:
### Find the Root Mean Squared Error.
RMSE_LSTM = math.sqrt(np.square(np.subtract(y_test_2, y_pred_lstm)).mean())
print('RMSE - LSTM: ' + str(RMSE_LSTM))

RMSE - LSTM: 0.035317970425821486


Long Short-Term Memory Model (LSTM - 2 hidden layers)

In [130]:
### Create LSTM model with the use of a function
def build_lstm_2(hp):
  hp_units = hp.Int('units', min_value = 10, max_value = 100, step = 10)
  hp_learning_rate = hp.Choice('learning_rate',
                               values = [1e-1, 1e-2, 1e-3, 1e-4])
  hp_activation = hp.Choice('activation',
                            values = ['relu', 'tanh', 'sigmoid'])
  model = tf.keras.Sequential()
  model.add((keras.layers.LSTM(units = hp_units,
                               activation = hp_activation,
                               input_shape = (X_train_2.shape[1],
                                              X_train_2.shape[2]),
                               return_sequences = True)))
  model.add((keras.layers.LSTM(units = hp_units,
                               activation = hp_activation,
                               return_sequences = True)))
  model.add(keras.layers.Dense(y_train_2.shape[2]))

  model.compile(optimizer = Adam(learning_rate = hp_learning_rate),
                loss = 'mse',
                metrics = tf.keras.metrics.RootMeanSquaredError())
  return model

In [131]:
### Instantiate the tuner and perform hyperparameter tuning.
lstm_tuner_2 = kt.Hyperband(build_lstm_2,
                     objective = 'val_loss',
                     max_epochs = 10,
                     directory = 'keras_tuner_dir',
                     project_name = 'keras_lstm',
                     overwrite = True)

In [132]:
### Early Stopping definition.
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 5)

In [133]:
### Hyperparameters search for 5 epochs.
lstm_tuner_2.search(X_train_2, y_train_2,
             epochs = 5, validation_data = (X_val_2, y_val_2),
              callbacks = [stop_early])

Trial 30 Complete [00h 00m 04s]
val_loss: 0.0014521593693643808

Best val_loss So Far: 0.0013338770950213075
Total elapsed time: 00h 02m 36s
INFO:tensorflow:Oracle triggered exit


In [134]:
# Get the optimal hyperparameters.
lstm_best_hps_2 = lstm_tuner_2.get_best_hyperparameters(num_trials=1)[0]

In [135]:
print(f"""
The hyperparameter search for LSTM (2 layers) is complete. 
The optimal number of units is {lstm_best_hps_2.get('units')},
the optimal activation is {lstm_best_hps_2.get('activation')}
and the optimal learning rate for the optimizer
is {lstm_best_hps_2.get('learning_rate')}
""")


The hyperparameter search for LSTM (2 layers) is complete. 
The optimal number of units is 100,
the optimal activation is tanh
and the optimal learning rate for the optimizer
is 0.01



In [136]:
### Find the optimal number of epochs to train the model
### with the hyperparameters obtained from the search.
### Build the model with the optimal hyperparameters
### and train it on the data for 100 epochs.
model = lstm_tuner_2.hypermodel.build(lstm_best_hps_2)

history = model.fit(X_train_2, y_train_2,
                    epochs=100, validation_data = (X_val_2, y_val_2))

val_rmse_per_epoch = history.history['val_root_mean_squared_error']

best_epoch = val_rmse_per_epoch.index(min(val_rmse_per_epoch))

print('Best epoch: %d' % (best_epoch,))

Epoch 1/100
5/5 [==============================] - 5s 238ms/step - loss: 0.0040 - root_mean_squared_error: 0.0632 - val_loss: 0.0015 - val_root_mean_squared_error: 0.0387
Epoch 2/100
5/5 [==============================] - 0s 12ms/step - loss: 0.0031 - root_mean_squared_error: 0.0556 - val_loss: 0.0014 - val_root_mean_squared_error: 0.0377
Epoch 3/100
5/5 [==============================] - 0s 11ms/step - loss: 0.0028 - root_mean_squared_error: 0.0531 - val_loss: 0.0015 - val_root_mean_squared_error: 0.0390
Epoch 4/100
5/5 [==============================] - 0s 12ms/step - loss: 0.0027 - root_mean_squared_error: 0.0515 - val_loss: 0.0014 - val_root_mean_squared_error: 0.0376
Epoch 5/100
5/5 [==============================] - 0s 12ms/step - loss: 0.0025 - root_mean_squared_error: 0.0502 - val_loss: 0.0014 - val_root_mean_squared_error: 0.0371
Epoch 6/100
5/5 [==============================] - 0s 14ms/step - loss: 0.0023 - root_mean_squared_error: 0.0483 - val_loss: 0.0015 - val_root_mean_s

In [137]:
### Pass the optimal hyperparameters to hypermodel.
hypermodel_2_2 = lstm_tuner_2.hypermodel.build(lstm_best_hps_2)

### Retrain the model with the optimal number of epochs.
hypermodel_2_2.fit(X_train_2, y_train_2,
                 epochs = best_epoch,
                 validation_data = (X_val_2, y_val_2),
                 shuffle = False)

Epoch 1/16
5/5 [==============================] - 5s 238ms/step - loss: 0.0038 - root_mean_squared_error: 0.0616 - val_loss: 0.0015 - val_root_mean_squared_error: 0.0383
Epoch 2/16
5/5 [==============================] - 0s 12ms/step - loss: 0.0031 - root_mean_squared_error: 0.0553 - val_loss: 0.0014 - val_root_mean_squared_error: 0.0376
Epoch 3/16
5/5 [==============================] - 0s 12ms/step - loss: 0.0028 - root_mean_squared_error: 0.0527 - val_loss: 0.0014 - val_root_mean_squared_error: 0.0378
Epoch 4/16
5/5 [==============================] - 0s 12ms/step - loss: 0.0026 - root_mean_squared_error: 0.0515 - val_loss: 0.0014 - val_root_mean_squared_error: 0.0368
Epoch 5/16
5/5 [==============================] - 0s 12ms/step - loss: 0.0024 - root_mean_squared_error: 0.0486 - val_loss: 0.0014 - val_root_mean_squared_error: 0.0370
Epoch 6/16
5/5 [==============================] - 0s 12ms/step - loss: 0.0022 - root_mean_squared_error: 0.0465 - val_loss: 0.0014 - val_root_mean_squared

In [138]:
### Predicted output values using X_test_2.
y_pred_lstm_2 = hypermodel_2_2.predict(X_test_2)

In [139]:
### Evalaute hypermodel using test sets.
eval_result_lstm = hypermodel_2_2.evaluate(X_test_2, y_test_2)
print("[test loss, test mse]:", eval_result_lstm)

2/2 [==============================] - 2s 8ms/step - loss: 0.0013 - root_mean_squared_error: 0.0364
[test loss, test mse]: [0.0013238996034488082, 0.0363854318857193]


In [140]:
### Find the Root Mean Squared Error
RMSE_LSTM_2 = math.sqrt(np.square(np.subtract(y_test_2, y_pred_lstm_2)).mean())
print('RMSE - LSTM 2: ' + str(RMSE_LSTM_2))

RMSE - LSTM 2: 0.036385431898619106


Mean Variance Portfolio Optimization

Historical Portfolio

In [141]:
### Financial variables
NAESX = pd.read_csv('NAESX.csv', sep = ',', index_col = 0)
NAESX.index = pd.to_datetime(NAESX.index)
NAESX = NAESX.drop(columns = ['Open', 'High', 'Low', 'Close', 'Volume'])
NAESX.columns = ['NAESX']

VBMFX = pd.read_csv('VBMFX.csv', sep = ',', index_col = 0)
VBMFX.index = pd.to_datetime(VBMFX.index)
VBMFX = VBMFX.drop(columns = ['Open', 'High', 'Low', 'Close', 'Volume'])
VBMFX.columns = ['VBMFX']

VEIEX = pd.read_csv('VEIEX.csv', sep = ',', index_col = 0)
VEIEX.index = pd.to_datetime(VEIEX.index)
VEIEX = VEIEX.drop(columns = ['Open', 'High', 'Low', 'Close', 'Volume'])
VEIEX.columns = ['VEIEX']

VFINX = pd.read_csv('VFINX.csv', sep = ',', index_col = 0)
VFINX.index = pd.to_datetime(VFINX.index)
VFINX = VFINX.drop(columns = ['Open', 'High', 'Low', 'Close', 'Volume'])
VFINX.columns = ['VFINX']

VGSLX = pd.read_csv('VGSLX.csv', sep = ',', index_col = 0)
VGSLX.index = pd.to_datetime(VGSLX.index)
VGSLX = VGSLX.drop(columns = ['Open', 'High', 'Low', 'Close', 'Volume'])
VGSLX.columns = ['VGSLX']

VGTSX = pd.read_csv('VGTSX.csv', sep = ',', index_col = 0)
VGTSX.index = pd.to_datetime(VGTSX.index)
VGTSX = VGTSX.drop(columns = ['Open', 'High', 'Low', 'Close', 'Volume'])
VGTSX.columns = ['VGTSX']

mutual_funds = [NAESX, VBMFX, VEIEX, VFINX, VGSLX, VGTSX]
mutual_funds_prices = reduce(lambda left, right : pd.merge(left, right,
                                         left_index = True, right_index = True,
                                         how = 'outer'), mutual_funds)
mutual_funds_prices = mutual_funds_prices['2002-01-01' : '2019-12-01']
mutual_funds_prices = mutual_funds_prices.fillna(0.0)
mutual_funds_returns = mutual_funds_prices.pct_change().apply(lambda x: np.log(1+x))

In [142]:
### Find mean monthly return and covariance of monthly returns.
mean_monthly_returns = mutual_funds_returns.mean()
cov_matrix = mutual_funds_returns.cov()
 
### Assuming portfolio weights are equally set.
weights = np.asarray([1/6, 1/6, 1/6, 1/6, 1/6, 1/6])
 
portfolio_return = round(np.sum(mean_monthly_returns * weights) * 12,2)
portfolio_std_dev = round(np.sqrt(np.dot(weights.T,np.dot(cov_matrix, weights))) * np.sqrt(12),2)

print("Expected annualised return: " + str(portfolio_return))
print("Volatility: " + str(portfolio_std_dev))

Expected annualised return: 0.08
Volatility: 0.14


In [143]:
### Portfolio optimization, minimizing volatility.
### The output is an array of weights for each stock.

### Expected returns and sample covariance.
mu = expected_returns.mean_historical_return(mutual_funds_prices)
S = risk_models.sample_cov(mutual_funds_prices)

# Optimise portfolio, minimizing volatility.
ef = EfficientFrontier(mu, S)
raw_weights = ef.min_volatility()
cleaned_weights = ef.clean_weights()
print(cleaned_weights)
ef.portfolio_performance(verbose=True)

OrderedDict([('NAESX', 0.00776), ('VBMFX', 0.9267), ('VEIEX', 0.0), ('VFINX', 0.06554), ('VGSLX', 0.0), ('VGTSX', 0.0)])
Expected annual return: 154.5%
Annual volatility: 15.1%
Sharpe Ratio: 10.10


(1.545380301070697, 0.15101006155691746, 10.101183228084198)

In [144]:
### Weights' allocation to discrete values, showing the number of shares
### of each mutual fund index an investor should buy in order to minimize
### volatility.
from pypfopt import  discrete_allocation
from pypfopt.discrete_allocation import  DiscreteAllocation
latest_prices = discrete_allocation.get_latest_prices(mutual_funds_prices)
allocation, leftover = discrete_allocation.DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=100000).lp_portfolio()
print(allocation)
print("Funds remaining: ${:.2f}".format(leftover))

{'NAESX': 9, 'VBMFX': 8662, 'VFINX': 23}
Funds remaining: $0.47


Predicted Portfolio

In [151]:
### Find the predicted values of LSTM model (1 hidden layer)
### and concatenate it into one array.
y_pred_lstm_tr = hypermodel_2.predict(X_train_2)
y_pred_lstm_val = hypermodel_2.predict(X_val_2)
y_pred_lstm_test = hypermodel_2.predict(X_test_2)
lstm_predicted_returns = np.concatenate((y_pred_lstm_tr,
                                         y_pred_lstm_val,
                                         y_pred_lstm_test), axis = 0)

array([[[ 0.02383673, -0.0052248 ,  0.01711401, -0.00722454,
         -0.00013061, -0.00104558]],

       [[ 0.01208617, -0.00188858,  0.00762723,  0.00899113,
          0.00364275,  0.00299242]],

       [[ 0.02814865, -0.00867448,  0.04551662,  0.02492215,
          0.01403879,  0.02414655]],

       ...,

       [[ 0.01373366,  0.00157472,  0.01031494,  0.01060112,
          0.01836428,  0.01028183]],

       [[ 0.01385377,  0.00131663,  0.012086  ,  0.01071692,
          0.01693846,  0.01046304]],

       [[ 0.0139853 ,  0.00088097,  0.01245499,  0.0109985 ,
          0.0164239 ,  0.01021497]]], dtype=float32)

In [152]:
### Reshape the predicted array.
lstm_predicted_returns = lstm_predicted_returns.reshape((216,6))

array([[ 0.02383673, -0.0052248 ,  0.01711401, -0.00722454, -0.00013061,
        -0.00104558],
       [ 0.01208617, -0.00188858,  0.00762723,  0.00899113,  0.00364275,
         0.00299242],
       [ 0.02814865, -0.00867448,  0.04551662,  0.02492215,  0.01403879,
         0.02414655],
       ...,
       [ 0.01373366,  0.00157472,  0.01031494,  0.01060112,  0.01836428,
         0.01028183],
       [ 0.01385377,  0.00131663,  0.012086  ,  0.01071692,  0.01693846,
         0.01046304],
       [ 0.0139853 ,  0.00088097,  0.01245499,  0.0109985 ,  0.0164239 ,
         0.01021497]], dtype=float32)

In [153]:
### Remove standardization effects.
### NOTE: The mean and standard deviation are so small,
### that no changes are observed after removal. 
lstm_predicted_returns = lstm_predicted_returns * X_train.std() + X_train.mean()

array([[ 0.02383673, -0.0052248 ,  0.01711401, -0.00722454, -0.00013061,
        -0.00104558],
       [ 0.01208617, -0.00188858,  0.00762723,  0.00899113,  0.00364275,
         0.00299242],
       [ 0.02814865, -0.00867448,  0.04551662,  0.02492215,  0.01403879,
         0.02414655],
       ...,
       [ 0.01373366,  0.00157472,  0.01031494,  0.01060112,  0.01836428,
         0.01028183],
       [ 0.01385377,  0.00131663,  0.012086  ,  0.01071692,  0.01693846,
         0.01046304],
       [ 0.0139853 ,  0.00088097,  0.01245499,  0.0109985 ,  0.0164239 ,
         0.01021497]], dtype=float32)

In [154]:
### Convert array to dataframe.
lstm_predicted_returns_df = pd.DataFrame(lstm_predicted_returns,
                                         columns = ['NAESX', 'VBMFX',
                                                    'VEIEX', 'VFINX',
                                                    'VGSLX', 'VGTSX'])

        NAESX     VBMFX     VEIEX     VFINX     VGSLX     VGTSX
0    0.023837 -0.005225  0.017114 -0.007225 -0.000131 -0.001046
1    0.012086 -0.001889  0.007627  0.008991  0.003643  0.002992
2    0.028149 -0.008674  0.045517  0.024922  0.014039  0.024147
3    0.010194  0.003750  0.017355  0.007872  0.014285  0.015250
4    0.032493  0.000048  0.049856  0.021130  0.031813  0.038386
..        ...       ...       ...       ...       ...       ...
211  0.012089  0.000757  0.005544  0.008934  0.014135  0.005248
212  0.013805  0.000928  0.011013  0.010751  0.016135  0.009311
213  0.013734  0.001575  0.010315  0.010601  0.018364  0.010282
214  0.013854  0.001317  0.012086  0.010717  0.016938  0.010463
215  0.013985  0.000881  0.012455  0.010998  0.016424  0.010215

[216 rows x 6 columns]


In [155]:
### Find mean monthly return and covariance of monthly returns.
mean_monthly_returns = lstm_predicted_returns_df.mean()
cov_matrix = lstm_predicted_returns_df.cov()
 
### Assuming portfolio weights are equally set.
weights = np.asarray([1/6, 1/6, 1/6, 1/6, 1/6, 1/6])
 
portfolio_return = round(np.sum(mean_monthly_returns * weights) * 12,2)
portfolio_std_dev = round(np.sqrt(np.dot(weights.T,np.dot(cov_matrix, weights))) * np.sqrt(12),2)

print("Expected annualised return: " + str(portfolio_return))
print("Volatility: " + str(portfolio_std_dev))

Expected annualised return: 0.09
Volatility: 0.08


In [156]:
### Transform from logarithmic returns to values.
### Use actual initial values as initial values in the process of transformation.
lstm_predicted_returns_df = (lstm_predicted_returns_df + 1).cumprod()
lstm_predicted_returns_df['NAESX'] = mutual_funds_prices['NAESX'][0] * lstm_predicted_returns_df['NAESX']
lstm_predicted_returns_df['VBMFX'] = mutual_funds_prices['VBMFX'][0] * lstm_predicted_returns_df['VBMFX']
lstm_predicted_returns_df['VEIEX'] = mutual_funds_prices['VEIEX'][0] * lstm_predicted_returns_df['VEIEX']
lstm_predicted_returns_df['VFINX'] = mutual_funds_prices['VFINX'][0] * lstm_predicted_returns_df['VFINX']
lstm_predicted_returns_df['VGSLX'] = mutual_funds_prices['VGSLX'][0] * lstm_predicted_returns_df['VGSLX']
lstm_predicted_returns_df['VGTSX'] = mutual_funds_prices['VGTSX'][0] * lstm_predicted_returns_df['VGTSX']

,NAESX,VBMFX,VEIEX,VFINX,VGSLX,VGTSX
0,15.725772,5.166609,5.564115,71.949310,25.507135,5.310537
1,15.915836,5.156851,5.606554,72.596214,25.600056,5.326429
2,16.363844,5.112118,5.861745,74.405464,25.959450,5.455044
3,16.530659,5.131287,5.963476,74.991150,26.330286,5.538235
4,17.067799,5.131534,6.260792,76.575737,27.167933,5.750826
...,...,...,...,...,...,...
211,121.480919,8.872168,27.710264,303.784546,162.364639,22.876333
212,123.158028,8.880399,28.015442,307.050690,164.984375,23.089346
213,124.849434,8.894383,28.304420,310.305786,168.014206,23.326744
214,126.579063,8.906095,28.646509,313.631317,170.860092,23.570814


In [157]:
### Portfolio optimization, minimizing volatility.
### The output is an array of weights for each stock.

### Expected returns and sample covariance.
mu = expected_returns.mean_historical_return(lstm_predicted_returns_df)
S = risk_models.sample_cov(lstm_predicted_returns_df)

### Optimise portfolio, minimizing volatility.
ef = EfficientFrontier(mu, S)
raw_weights = ef.min_volatility()
cleaned_weights = ef.clean_weights()
print(cleaned_weights)
ef.portfolio_performance(verbose=True)

OrderedDict([('NAESX', 0.20128), ('VBMFX', 0.77627), ('VEIEX', 0.0), ('VFINX', 0.02245), ('VGSLX', 0.0), ('VGTSX', 0.0)])
Expected annual return: 295.7%
Annual volatility: 12.5%
Sharpe Ratio: 23.49


(2.956547866266688, 0.1250344458745462, 23.485910988185488)

In [158]:
### Weights' allocation to discrete values, showing the number of shares
### of each mutual fund index an investor should buy in order to minimize
### volatility.

latest_prices = discrete_allocation.get_latest_prices(lstm_predicted_returns_df)
allocation, leftover = discrete_allocation.DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=100000).lp_portfolio()
print(allocation)
print("Funds remaining: ${:.2f}".format(leftover))

{'NAESX': 157, 'VBMFX': 8708, 'VFINX': 7}
Funds remaining: $7.00
